In [1]:
from dotenv import load_dotenv
load_dotenv()

# get to working directory of llms
import sys
sys.path.append('/pub/anvieyra/copilot/llms/user_modules')
from db import Session, get_training_data, add_sentiments_data
from helper_functions import generated_decoded_output, find_sentiment
import pandas as pd

In [2]:
# get training data for sentiment analysis
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [3]:
unproc_orca = training_data.apply(generated_decoded_output, axis = 1, model_id = "microsoft/Orca-2-7b")
unproc_orca.head()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment,generated_sentiment,memory_sentiment_usage,time_sentiment_taken
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral,"Given the following dialogue, output a single ...",26041.963379,0.353360
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive,"Given the following dialogue, output a single ...",26204.654785,0.270215
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative,"Given the following dialogue, output a single ...",26202.532715,0.269059
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive,"Given the following dialogue, output a single ...",26185.696777,0.270434
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive,"Given the following dialogue, output a single ...",26041.932129,0.200529


In [6]:
unproc_orca.to_csv("orca-sentiment.csv")

In [7]:
proc_orca = unproc_orca.apply(find_sentiment, axis = 1)

In [9]:
proc_orca["model_id"] = "orca2 7b"

In [10]:
proc_orca = proc_orca[["dialogue_id", "model_id", "generated_sentiment", "memory_sentiment_usage", "time_sentiment_taken"]]

In [11]:
add_sentiments_data(proc_orca)